#### Create a df with all information (elecs, max da, power when da is max)

In [1]:
"429 electrodes included"

'429 electrodes included'

In [5]:
from brainpipe.system import study
from os.path import isfile, join
import numpy as np
from itertools import product
import pandas as pd

conds = ['low','high']
rep = 'Encoding/' #'Retrieval_new_odors/', 'Retrieval_new_rec/'
st = study('Ripples')
path_npz = join(st.path, 'classified/'+rep)
f_form = 'All_subjects_sources_{}_{}_{}.npz'
f_form = join(path_npz, f_form)
dfname = '0_all_subjects_info_elecs.csv'
df_all_name = '1_all_subjects_AUC_Pow_Time_bsl_all_freqs.csv'
###############################################################################
freqs = ['HFA','ripple','low_freq']
data_type = 'pow'
###############################################################################

mat = np.load(f_form.format(freqs[0],conds[0],conds[1]),allow_pickle=True)
subjects = mat['su_codes'][:,np.newaxis]
elecs = mat['s_elec'][:,np.newaxis]
s_Mai_RL = mat['s_labels'][:,np.newaxis]
channels = mat['s_channels'][:,np.newaxis]
elec_data = np.concatenate((subjects, s_Mai_RL,channels,elecs), axis=1)
df0 = pd.DataFrame(elec_data, columns=['subjects','s_Mai_RL','channels','elecs_num'])
df0.to_csv(path_npz+dfname)

# load all electrodes information and create a df
idx_all, pow0_all, pow1_all, da_all = np.array([]),np.array([]),np.array([]),np.array([])
for freq in freqs:
    mat = np.load(f_form.format(freq,conds[0],conds[1]),allow_pickle=True)
    idx_elecs, pow0_elecs, pow1_elecs, da_elecs = np.array([]),np.array([]),np.array([]),np.array([])
    for elec in range(elecs.shape[0]):
        da = mat['s_da'][elec]
        val0 = 's_elec_pow0' if data_type == 'pow' else 'rsa0'
        val1 = 's_elec_pow1' if data_type == 'pow' else 'rsa1'
        if np.size(da) > 1:
            idx_elec = [i for i,j in enumerate(da) if j == max(da)][0]
            da_elec = np.max(da)
            pow0_elec = np.mean(mat[val0][elec][idx_elec])
            pow1_elec = np.mean(mat[val1][elec][idx_elec])
        else:
            da_elec = da
            pow0_elec = mat[val0][elec]
            pow1_elec = mat[val1][elec]
        pow0_elecs = np.hstack((pow0_elecs,pow0_elec)) if np.size(pow0_elecs) else pow0_elec
        pow1_elecs = np.hstack((pow1_elecs, pow1_elec)) if np.size(pow1_elecs) else pow1_elec
        idx_elecs = np.hstack((idx_elecs, idx_elec)) if np.size(idx_elecs) else idx_elec
        da_elecs = np.hstack((da_elecs,da_elec)) if np.size(da_elecs) else da_elec
    idx_all = np.vstack((idx_all, idx_elecs)) if np.size(idx_all) else idx_elecs
    pow0_all= np.vstack((pow0_all, pow0_elecs)) if np.size(pow0_all) else pow0_elecs
    pow1_all= np.vstack((pow1_all, pow1_elecs)) if np.size(pow1_all) else pow1_elecs
    da_all = np.vstack((da_all, da_elecs)) if np.size(da_all) else da_elecs
df2 = pd.DataFrame(da_all.T, columns=[freq+'_AUC' for freq in freqs])
df3 = pd.DataFrame(pow0_all.T, columns=[freq+'_Pow0' for freq in freqs])
df4 = pd.DataFrame(pow1_all.T, columns=[freq+'_Pow1' for freq in freqs])
df5 = pd.DataFrame(idx_all.T, columns=[freq+'_Time' for freq in freqs])
df_all = pd.concat((df0,df2,df3,df4),axis=1)
df_all.to_csv(path_npz+df_all_name)

-> Ripples loaded


#### create statistical masks correction by subject (elecs, time, freqs) or by roi (subjects, time,freqs)

In [9]:
"""
Correct statistics by region for all freqs, electrodes and subjects
"""
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
from brainpipe.statistics import *
import numpy as np
from itertools import product
import seaborn as sns
import pandas as pd

conds = ['low','high']
rep = 'Encoding/'
###############################################################################
st = study('Ripples')
path_npz = join(st.path, 'classified/'+rep)
path_to_save = join(path_npz, 'masks_stat/')
f_form = 'All_subjects_sources_{}_'+conds[0]+'_'+conds[1]+'.npz'
f_form = join(path_npz, f_form)
###############################################################################
if not exists(path_to_save):
    makedirs(path_to_save)
###############################################################################  
subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','VACJ']
freqs = ['HFA','ripple','low_freq']
#rois_to_keep = ['ACC','aHC','pHC','IFG','Ins_olf','MFG','OFC_olf','PHG','SFG','pPirT']
rois_to_keep = ['CD', 'Frontal', 'INS', 'MTL', 'OFC_OLF', 'PHG_FuG', 'Parietal', 'aMTL',
                'cingulate', 'occipital', 'olf', 'orbital', 'temp_cx']
thrs = [0.05,0.01]
n_perms = 100

def find_th_perm(perms, thr, maxst=True):
    if perms.shape[1] > 0:
        th_perm = perm_pvalue2level(perms, p=thr, maxst=maxst)[0]
    else:
        perms = perms[:,np.newaxis]
        th_perm = perm_pvalue2level(perms, p=thr, maxst=maxst)[0]
    return th_perm

# #Find threshold by subject
for t in thrs:
    th_roi = []
    for roi in rois_to_keep:
        th_freq_roi = []
        for freq in freqs:
            #print(roi)
            mat = np.load(f_form.format(freq),allow_pickle=True)
            id_lab = [i for i,x in enumerate(mat['s_labels']) if x == roi]
            #print(id_lab)
            s_perm = mat['s_perm'][id_lab].swapaxes(0,-1)
            th_perm = find_th_perm(s_perm,t)
            th_freq_roi.append(th_perm)
        th_roi.append(np.max(th_freq_roi))
    print('perm max size',t, th_roi)
    
    th_by_su = []
    for su in sorted(subjects):
        th_su = []
        for freq in freqs:
            mat = np.load(f_form.format(freq),allow_pickle=True)
            id_su = np.where(mat['su_codes']==su)[0]
            id_lab = [i for i,x in enumerate(mat['s_labels'][id_su]) if x in rois_to_keep]
            s_perm = mat['s_perm'][id_su][id_lab].swapaxes(0,-1)
            th_perm = find_th_perm(s_perm,t)
            th_su.append(th_perm)
        th_by_su.append(np.max(th_su))
    print('perm max size',th_by_su)

    df = pd.read_csv(path_npz+'1_all_subjects_AUC_Pow_Time_bsl_all_freqs.csv')
    df_sel = df.loc[df['s_Mai_RL'].isin(rois_to_keep)]
    dic_th_roi = {}
    for i,roi in enumerate(rois_to_keep):
        dic_th_roi[roi] = th_roi[i]
    dic_th_su = {}
    for i,su in enumerate(sorted(subjects)):
        dic_th_su[su] = th_by_su[i]
    df_sel['thr_roi'] = df_sel['s_Mai_RL'].map(dic_th_roi)
    df_sel['thr_subj'] = df_sel['subjects'].map(dic_th_su)
    for freq in freqs:
        #1 if sig 0 if not
        df_sel['roi_sig_'+freq] = [0 if auc <= th else 1 for auc, th in zip(df_sel[freq+'_AUC'],df_sel['thr_roi'])]
        df_sel['su_sig_'+freq] = [0 if auc <= th else 1 for auc, th in zip(df_sel[freq+'_AUC'],df_sel['thr_subj'])]
    df_sel.to_csv(path_npz+'2_all_subjects_AUC_by_roi_subj_'+str(t)+'_bsl_corr.csv')

-> Ripples loaded
perm max size 0.05 [0.9333333333333333, 1.0, 0.9, 0.8555555555555557, 0.9333333333333332, 0.9, 1.0, 0.9, 0.9333333333333333, 0.7214285714285714, 0.9, 0.9, 0.9277777777777778]
perm max size [1.0, 0.85, 0.740952380952381, 0.9, 0.8428571428571429, 0.9277777777777778]
perm max size 0.01 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.849047619047619, 1.0, 1.0, 1.0]
perm max size [1.0, 1.0, 0.8871428571428572, 1.0, 0.9833333333333334, 1.0]


#### sum up classif results 

In [126]:
from brainpipe.system import study
from os.path import join, exists
from os import makedirs
import numpy as np
from itertools import product
import pandas as pd

rep, thrs = 'Encoding/', ['0.05']
conds = ['Poor', 'Rich']
freqs = ['rsa_diag']
rois_olf = ['OFC_olf','pPirT','Amg']
data_type = 'pow'

st = study('Ripples')
pathdata = join(st.path, 'figure/0_clf_rsa_diag_sklearn_time_'+exp+'_by_cond/')
dfname = join(pathdata, '2_all_subjects_info_elecs_AUC_by_roi_subj_{}_bsl_corr.csv')
path_classif = join(st.path, 'classified/RSA_clf_diag_time_'+exp+'_by_cond/')
clf_name = join(path_classif, '{}_LDA_clf_rsa_diag_low_high.npz')
#clf_name = join(path_classif, '{}_LDA_clf_{}_{}_{}.npz')
#path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_cond/')
#pow_file = join(path_pow, '{}_odor_{}_{}_bsl_6freqs.npz')
path_pow = join(st.path,'feature/1_RSA_diag_'+exp+'_by_cond/')
pow_file = join(path_pow, 'rsa_diag_{}_cond_{}.npz')
path2save = join(pathdata, 'figs_{}/')
save_name = join(path2save, '{}_LDA_clf_{}_{}_{}_{}.png') #su,freq,thr,lab,num

def sum_up_clf(rois,df_sig,method,freq,th,plot=False):
    for roi in rois:
        df_sig_roi = df_sig.loc[df_sig['s_Mai_RL'] == roi]
        df_sig_roi['sign'] = np.sign(df_sig_roi[freq+'_Pow1'] - df_sig_roi[freq+'_Pow0'])
        inc = (df_sig_roi.loc[df_sig_roi.sign == 1.0]).shape[0]
        dec = (df_sig_roi.loc[df_sig_roi.sign == -1.0]).shape[0]
        n_su_inc = (df_sig_roi.loc[df_sig_roi.sign == 1.0].groupby(['subjects']).count()).shape[0]
        n_su_dec = (df_sig_roi.loc[df_sig_roi.sign == -1.0].groupby(['subjects']).count()).shape[0]
        
        #print(roi,' %s electrodes showed increased TPSim, while %s showed decrease out of %s elecs' 
        #          % (inc,dec,inc+dec))
        #print('NB of subjects with increase',n_su_inc)
        #print('NB of subjects with decrease',n_su_dec)
            
        if (n_su_inc >= 3) or (n_su_inc >=2 and roi in rois_olf):
            print(th,' \n>>SUM UP {}'.format(method))
            print('>>> for %s in %s, %s subjects have sig elecs' % (roi,freq,n_su_inc))
            df_inc = df_sig_roi.loc[df_sig_roi.sign == 1.0]
            auc = df_inc[freq+'_AUC'].values
            auc_mean, std_auc = np.mean(auc), np.std(auc)
            print('AUC decoding score is %s in average +/- %s' %(round(auc_mean,2),
                            round(std_auc,2)))
            print(df_inc)
            if plot == True:
                plot_save(df_inc,freq,exp,th,data_type)
            #plot_save(df_inc)
        
        if (n_su_dec >= 3) or (n_su_dec >=2 and roi in rois_olf):
            print(th,'\n>>SUM UP {}'.format(method))
            print('>>> for %s in %s, %s subjects have sig elecs' % (roi,freq,n_su_dec))
            df_dec = df_sig_roi.loc[df_sig_roi.sign == -1.0]
            auc = df_dec[freq+'_AUC'].values
            auc_mean, std_auc = np.mean(auc), np.std(auc)
            print('AUC decoding score is %s in average +/- %s' %(round(auc_mean,2),
                            round(std_auc,2)))
            print(df_dec)
            if plot == True:
                plot_save(df_dec,freq,exp,th,data_type)
            
for thr in thrs:
    df = pd.read_csv(dfname.format(thr))
    df_ = df[['subjects','s_Mai_RL','channels','elecs_num','thr_roi','thr_subj']]
    for freq in freqs:
        df_freq = pd.concat([df_,df.filter(like=freq)], axis=1)
        df_sig_roi = df_freq.loc[df_freq['roi_sig_'+freq[:]]>0]
        df_sig_su = df_freq.loc[df_freq['su_sig_'+freq[:]]>0]
        #print('Signif electrodes by roi', freq, len(df_sig_roi))
        #print('Signif electrodes by su', freq, len(df_sig_su))

        sig_rois = np.unique(df_sig_roi['s_Mai_RL'])
        sig_rois_su = np.unique(df_sig_su['s_Mai_RL'])
        #sum_up_clf(sig_rois, df_sig_roi, 'by roi',freq,thr,plot=False)
        sum_up_clf(sig_rois_su, df_sig_su, 'by subj',freq,thr)
        

-> Olfacto loaded
0.01  
>>SUM UP by subj
>>> for IFG in rsa_diag, 4 subjects have sig elecs
AUC decoding score is 0.63 in average +/- 0.03
    subjects s_Mai_RL   channels  elecs_num   thr_roi  thr_subj  rsa_diag_AUC  \
31      FERJ      IFG      o8-o7         33  0.609718  0.595032      0.660513   
64      LEFC      IFG    o11-o10         27  0.609718  0.569285      0.589047   
100     SEMC      IFG      u8-u7         28  0.609718  0.604084      0.619005   
184     VACJ      IFG  o'12-o'11         37  0.609718  0.612251      0.647020   

     rsa_diag_Pow0  rsa_diag_Pow1  roi_sig_rsa_diag  su_sig_rsa_diag  sign  
31       -0.124510       0.094859                 1                1   1.0  
64       -0.004079       0.076562                 0                1   1.0  
100       0.023598       0.175496                 1                1   1.0  
184      -0.112442       0.043888                 1                1   1.0  
0.01 
>>SUM UP by subj
>>> for Ins in rsa_diag, 3 subjects have sig e

In [102]:
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
from brainpipe.visual import *

def plot_save(df,freq,exp,th,data_type):
    subjects = np.unique(df['subjects'])
    thr_perms = df['thr_subj'].values[0]
    i_freq = [i for i,f in enumerate(freqs) if f==freq][0]
    for su in subjects:
        list_elecs = df.loc[df['subjects']==su]['channels'].values
        mat = np.load(pow_file.format(su,'low',exp),allow_pickle=True)
        for chan in list_elecs:
            idx_elec = [i for i,ch in enumerate(mat['channels']) if ch==chan][0]
            if data_type == 'pow':
                pow0, lab = mat['xpow'][i_freq,idx_elec,17:47,:], mat['labels'][idx_elec]
                pow1 = np.load(pow_file.format(su,'high',exp))['xpow'][i_freq,idx_elec,17:47,:]
                time = mat['time'][17:47]
            if data_type == 'rsa':
                pow0, lab = mat['rsa'][idx_elec,:].swapaxes(0,1), mat['labels'][idx_elec]
                pow1 = np.load(pow_file.format(su,'high',exp))['rsa'][idx_elec,:].swapaxes(0,1)
                time = mat['time']
            print(pow0.shape,pow1.shape,time.shape)
            y = np.array([0]*pow0.shape[-1]+[1]*pow1.shape[-1])
            x = np.concatenate((pow0,pow1), axis=-1)
            da = np.load(clf_name.format(su,freq,'low','high'))['auc'][idx_elec,:][:,np.newaxis]

            #plot and figure parameters
            xfmt = ScalarFormatter(useMathText=True)
            xfmt.set_powerlimits((0,3))
            fig = plt.figure(1,figsize=(7,7))
            title = su+' '+conds[0]+' vs '+conds[1]+' '+str(lab)+' ('+str(chan)+')'
            fig.suptitle(title, fontsize=12)

            # Plot the POW + STATS
            plt.subplot(211)        
            BorderPlot(time, x, y=y, kind='sem', alpha=0.2, color=['darkorange','blue'],
                       linewidth=2, ncol=1, xlabel='Time (s)',ylabel = r'Power', 
                       legend=['low','high'])
            rmaxis(plt.gca(), ['right', 'top'])
            addLines(plt.gca(), vLines=[0], vColor=['darkgray'], vWidth=[2])
            plt.legend(loc=0, handletextpad=0.1, frameon=False)
            plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

            # Plot DA for the POW
            plt.subplot(212)
            BorderPlot(time, da, color='b', kind='sd',xlabel='Time (s)', ylim=[0.4,1.], 
                       ylabel='AUC score',linewidth=2, alpha=0.3)
            rmaxis(plt.gca(), ['right', 'top'])
            addLines(plt.gca(), vLines=[0], vColor=['darkgray'], vWidth=[2])
            plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
            plt.plot(time, thr_perms*np.ones(len(time)), '--', color='r', linewidth=2)

            #Save plots
            if not exists(path2save.format(th)):
                makedirs(path2save.format(th))
            plt.savefig(save_name.format(th,su,freq,th,lab,idx_elec),dpi=300,
                        bbox_inches='tight')
            plt.savefig(save_name.format(th,su,freq,th,lab,idx_elec).replace('png','pdf'),
                       dpi=300,bbox_inches='tight')
            plt.clf()
            plt.close()